In [1]:
import sys
sys.path.append('../../../')
from pprint import pprint
import time
import pickle
import importlib
import numpy as np

In [2]:
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from lib_grover import make_hit_list
import mthree
from qiskit.test.mock import FakeManhattan
from libs_qrem import sgs_algorithm_py

In [3]:
nbit = 20 # change this value to get discretized result closer to analytical results # 3
#setting the number of shots and Grover operators.
max_iter = 10

In [4]:
with open("pkls/raw_hist_list_list.pkl", "rb") as f:
    raw_hist_list_list = pickle.load(f)
with open("pkls/meas_fitters.pkl", "rb") as f:
    meas_fitters = pickle.load(f)

In [5]:
mthree_lu_mitigator_list = []
for i in range(0, max_iter):
    mit = mthree.M3Mitigation(FakeManhattan())
    mit.single_qubit_cals = meas_fitters[i].cal_matrices
    mthree_lu_mitigator_list.append(mit)
    if i % 10 == 0:
        print("size", i, "finished")
print("length of mthree_lu_mitigator_list: ", len(mthree_lu_mitigator_list))

size 0 finished
length of mthree_lu_mitigator_list:  10


In [6]:
mthree_lu_info_list_list = []
mthree_lu_hit_list_list = []
for i, raw_hist_list in enumerate(raw_hist_list_list):
    mthree_lu_info_list = []
    mthree_lu_hist_list = []
    for raw_hist in raw_hist_list:
        meas_layout = meas_layout = [j for j in range(nbit + 1)]
        mit_hist, info = mthree_lu_mitigator_list[i].apply_correction(raw_hist,
                                                                      meas_layout, 
                                                                      distance=None,
                                                                      method='direct',
                                                                      max_iter=25, 
                                                                      tol=1e-5,
                                                                      return_mitigation_overhead=True,
                                                                      details=True)
        mthree_lu_hist_list.append( sgs_algorithm_py(mit_hist) )
        mthree_lu_info_list.append( info )
    mthree_lu_hit_list_list.append( make_hit_list(mthree_lu_hist_list) )
    mthree_lu_info_list_list.append( mthree_lu_info_list )
    print("iter", i, "finished")

iter 0 finished
iter 1 finished
iter 2 finished
iter 3 finished
iter 4 finished
iter 5 finished
iter 6 finished
iter 7 finished
iter 8 finished
iter 9 finished


In [7]:
with open("pkls/mthree_lu_info_list_list.pkl", "wb") as f:
    pickle.dump(mthree_lu_info_list_list, f)
with open("pkls/mthree_lu_hit_list_list.pkl", "wb") as f:
    pickle.dump(mthree_lu_hit_list_list, f)